In [1]:
TRUTH_DIR = '/BrainSeg/ground_truth_masks/'
TEST_DIR = '/BrainSeg/Classify_Results/ResNet/GMBMasks/'
SAVE_FILENAME = 'AccuracyResults.csv' # Under TEST_DIR
SAVE_FILENAME_LATEX = 'AccuracyResults.txt' # Under TEST_DIR

In [2]:
import os
import glob
import numpy as np
import pyvips
from tqdm import tqdm
import re
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

In [3]:
!rm /BrainSeg/ground_truth_masks/.ipynb_checkpoints -rf

In [4]:
truth_images = sorted(os.listdir(TRUTH_DIR))
IMAGEDIMNORM = np.zeros((30,2), np.int)
for i in range(0, len(truth_images), 3):
    img = pyvips.Image.new_from_file(TRUTH_DIR + truth_images[i])
    width = img.width
    height = img.height
    IMAGEDIMNORM[i//3,0] = width
    IMAGEDIMNORM[i//3,1] = height
print(IMAGEDIMNORM)
downscale = 128
print(IMAGEDIMNORM.shape)

[[53784 47177]
 [73704 46200]
 [65736 45178]
 [67728 36459]
 [59760 41787]
 [55776 39849]
 [61752 46410]
 [57768 43740]
 [49800 47520]
 [57768 47481]
 [53784 45142]
 [71712 47426]
 [67728 47200]
 [47808 41668]
 [49800 36131]
 [53784 45548]
 [53784 41073]
 [55776 41634]
 [59760 44874]
 [55776 44644]
 [47808 36870]
 [49800 43128]
 [47808 35729]
 [61752 38443]
 [47808 39068]
 [37848 37641]
 [39840 41543]
 [75695 77386]
 [55776 40419]
 [53784 43598]]
(30, 2)


In [8]:
53784/128

420.1875

In [5]:
IMAGEDIMNORM.shape

(30, 2)

In [6]:
image_names = glob.glob(TRUTH_DIR + "*Grey.png")
image_names = [imagename.split('/')[-1] for imagename in image_names]
image_names = [imagename.split('-Grey')[0] for imagename in image_names]
image_names = sorted(image_names)
print(len(image_names))

30


In [7]:
#@title
# Stores results: grey_TN, grey_FP, grey_FN, grey_TP, white_TN, white_FP, white_FN, white_TP
results = np.zeros((len(image_names), 8), dtype='int')

t = tqdm(total=len(image_names))
for i, imagename in enumerate(image_names):
    # Filename for testing and ground truth images
    test_grey_img_name = imagename + "-Grey.png"
    test_white_img_name = imagename + "-White.png"
    truth_grey_img_name = imagename + "-Grey.png"
    truth_white_img_name = imagename + "-White.png"

    ##### Testing Grey Matter Mask #####
    # Load images
    test_img = np.array(Image.open(TEST_DIR + test_grey_img_name))
    truth_img = Image.open(TRUTH_DIR + truth_grey_img_name)
    (width, height) = (IMAGEDIMNORM[i, 0] // downscale, IMAGEDIMNORM[i, 1] // downscale)
    truth_img = truth_img.resize((width, height), Image.BILINEAR)
    truth_img = np.array(truth_img)

    # Check dimensions
    if not (test_img.shape[0] == IMAGEDIMNORM[i, 1]//downscale and test_img.shape[1] == IMAGEDIMNORM[i, 0]//downscale):
        print('\t' + imagename + ' grey dimension WRONG!!!')

    # Update tqdm descriptions
    t.set_description_str("Image {}, Test ({}), Ground Truth ({})".format(
      imagename, str(test_img.shape), str(truth_img.shape)) )
    t.refresh()
    t.write("", end=' ')

    # Clip test_img to 1
    test_img[test_img>=1] = 1

    # Calculate true and false segmentations
    width = truth_img.shape[1]
    height = truth_img.shape[0]
    grey_TP = np.multiply(test_img, truth_img).sum()
    grey_FP = np.multiply(test_img, np.logical_not(truth_img)).sum()
    grey_FN = np.multiply(np.logical_not(test_img), truth_img).sum()
    grey_TN = np.multiply(np.logical_not(test_img), np.logical_not(truth_img)).sum()

    # Check results
    grey_total = width*height
    if not (grey_total == grey_TP+grey_FP+grey_FN+grey_TN):
        print('\t' + imagename + ' grey matter results WRONG!!!')

    ##### Testing White Matter Mask #####
    # Load images
    test_img = np.array(Image.open(TEST_DIR + test_white_img_name))
    truth_img = Image.open(TRUTH_DIR + truth_white_img_name)
    (width, height) = (IMAGEDIMNORM[i, 0] // downscale, IMAGEDIMNORM[i, 1] // downscale)
    truth_img = truth_img.resize((width, height), Image.BILINEAR)
    truth_img = np.array(truth_img)

    # Check dimensions
    if not (test_img.shape[0] == IMAGEDIMNORM[i, 1]//downscale and test_img.shape[1] == IMAGEDIMNORM[i, 0]//downscale):
        print('\t' + imagename + ' white dimension WRONG!!!')

    # Clip test_img to 1
    test_img[test_img>=1] = 1

    # Calculate true and false segmentations
    width = truth_img.shape[1]
    height = truth_img.shape[0]
    white_TP = np.multiply(test_img, truth_img).sum()
    white_FP = np.multiply(test_img, np.logical_not(truth_img)).sum()
    white_FN = np.multiply(np.logical_not(test_img), truth_img).sum()
    white_TN = np.multiply(np.logical_not(test_img), np.logical_not(truth_img)).sum()

    # Check results
    white_total = width*height
    if not (white_total == white_TN+white_FP+white_FN+white_TP):
        print('\t' + imagename + ' white matter results WRONG!!!')

    # Save results: grey_TN, grey_FP, grey_FN, grey_TP, white_TN, white_FP, white_FN, white_TP
    results[i, :] = [grey_TN, grey_FP, grey_FN, grey_TP, white_TN, white_FP, white_FN, white_TP]

    t.update()
t.close()

# Generate results file
with open(TEST_DIR + SAVE_FILENAME, "w") as f:
    f.write('Image Name,Grey_TN (px),Grey_FP (px),Grey_FN (px),Grey_TP (px),' +
          'White_TN (px),White_FP (px),White_FN (px),White_TP (px),' + 
          'G_Accuracy (%),G_Misclassification Rate (%),G_Sensitivity (%),' +
          'G_Specificity (%),G_Precision (%),G_Prevalence (%),' + 
          'W_Accuracy (%),W_Misclassification Rate (%),W_Sensitivity (%),' +
          'W_Specificity (%),W_Precision (%),W_Prevalence (%)\n') 
    accu_result = np.zeros((len(image_names), 12))
    for i, imagename in enumerate(image_names):
        total = results[i, 0]+results[i, 1]+results[i, 2]+results[i, 3]
        accu_result[i, 0]  = (results[i, 0]+results[i, 3])/total*100
        accu_result[i, 1]  = (results[i, 1]+results[i, 2])/total*100
        accu_result[i, 2]  = (results[i, 3]/(results[i, 2]+results[i, 3]))*100
        accu_result[i, 3]  = (results[i, 0]/(results[i, 0]+results[i, 1]))*100
        accu_result[i, 4]  = (results[i, 3]/(results[i, 1]+results[i, 3]))*100
        accu_result[i, 5]  = (results[i, 2]+results[i, 3])/total*100
        total = results[i, 4]+results[i, 5]+results[i, 6]+results[i, 7]
        accu_result[i, 6]  = (results[i, 4]+results[i, 7])/total*100
        accu_result[i, 7]  = (results[i, 5]+results[i, 6])/total*100
        accu_result[i, 8]  = (results[i, 7]/(results[i, 6]+results[i, 7]))*100
        accu_result[i, 9]  = (results[i, 4]/(results[i, 4]+results[i, 5]))*100
        accu_result[i, 10] = (results[i, 7]/(results[i, 5]+results[i, 7]))*100
        accu_result[i, 11] = (results[i, 6]+results[i, 7])/total*100
        f.write('%s,%d,%d,%d,%d,%d,%d,%d,%d,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f\n' % 
            (imagename, results[i, 0], results[i, 1], results[i, 2], results[i, 3], 
                results[i, 4], results[i, 5], results[i, 6], results[i, 7], 
                accu_result[i, 0], accu_result[i, 1], accu_result[i, 2], 
                accu_result[i, 3], accu_result[i, 4], accu_result[i, 5], 
                accu_result[i, 6], accu_result[i, 7], accu_result[i, 8], 
                accu_result[i, 9], accu_result[i, 10], accu_result[i, 11]))
    f.write('-' * 230 + '\n')
    f.write('Average,%d,%d,%d,%d,%d,%d,%d,%d,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f\n' % 
          (np.mean(results[:, 0]), np.mean(results[:, 1]), np.mean(results[:, 2]), np.mean(results[:, 3]), 
              np.mean(results[:, 4]), np.mean(results[:, 5]), np.mean(results[:, 6]), np.mean(results[:, 7]), 
              np.mean(accu_result[:, 0]), np.mean(accu_result[:, 1]), np.mean(accu_result[:, 2]), 
              np.mean(accu_result[:, 3]), np.mean(accu_result[:, 4]), np.mean(accu_result[:, 5]), 
              np.mean(accu_result[:, 6]), np.mean(accu_result[:, 7]), np.mean(accu_result[:, 8]), 
              np.mean(accu_result[:, 9]), np.mean(accu_result[:, 10]), np.mean(accu_result[:, 11])))

# # Generate results file for LaTeX
# with open(TEST_DIR + SAVE_FILENAME_LATEX, "w") as f:
#     # Grey Matter
#     f.write('Image Name,Grey_TN (px),Grey_FP (px),Grey_FN (px),Grey_TP (px),' +
#           'G_Accuracy (%),G_Sensitivity (%),G_Specificity (%),G_Precision (%)\n')
#     accu_result = np.zeros((len(image_names), 4))
#     for i, imagename in enumerate(image_names):
#     total = results[i, 0]+results[i, 1]+results[i, 2]+results[i, 3]
#     accu_result[i, 0]  = (results[i, 0]+results[i, 3])/total*100
#     accu_result[i, 1]  = (results[i, 3]/(results[i, 2]+results[i, 3]))*100
#     accu_result[i, 2]  = (results[i, 0]/(results[i, 0]+results[i, 1]))*100
#     accu_result[i, 3]  = (results[i, 3]/(results[i, 1]+results[i, 3]))*100
#     f.write('%s & %d & %d & %d & %d & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%%\\\\\n\\hline\n' % 
#         (imagename.split('-')[0], results[i, 0], results[i, 1], results[i, 2], results[i, 3], 
#             accu_result[i, 0], accu_result[i, 1], accu_result[i, 2], accu_result[i, 3]))
#     f.write('Average & %d & %d & %d & %d & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%%\\\\\n\n' % 
#           (np.mean(results[:, 0]), np.mean(results[:, 1]), np.mean(results[:, 2]), np.mean(results[:, 3]), 
#               np.mean(accu_result[:, 0]), np.mean(accu_result[:, 1]), 
#               np.mean(accu_result[:, 2]), np.mean(accu_result[:, 3])))

#     # White Matter
#     f.write('Image Name,White_TN (px),White_FP (px),White_FN (px),White_TP (px),' +
#           'W_Accuracy (%),W_Sensitivity (%),W_Specificity (%),W_Precision (%)\n')
#     accu_result = np.zeros((len(image_names), 4))
#     for i, imagename in enumerate(image_names):
#     total = results[i, 4]+results[i, 5]+results[i, 6]+results[i, 7] 
#     accu_result[i, 0]  = (results[i, 4]+results[i, 7])/total*100
#     accu_result[i, 1]  = (results[i, 7]/(results[i, 6]+results[i, 7]))*100
#     accu_result[i, 2]  = (results[i, 4]/(results[i, 4]+results[i, 5]))*100
#     accu_result[i, 3] = (results[i, 7]/(results[i, 5]+results[i, 7]))*100
#     f.write('%s & %d & %d & %d & %d & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%%\\\\\n\\hline\n' % 
#         (imagename.split('-')[0], results[i, 4], results[i, 5], results[i, 6], results[i, 7], 
#             accu_result[i, 0], accu_result[i, 1], accu_result[i, 2], accu_result[i, 3]))
#     f.write('Average & %d & %d & %d & %d & %.2f\\%% & %.2f\\%% & %.2f\\%% & %.2f\\%%\\\\\n\n' % 
#           (np.mean(results[:, 4]), np.mean(results[:, 5]), np.mean(results[:, 6]), np.mean(results[:, 7]), 
#               np.mean(accu_result[:, 0]), np.mean(accu_result[:, 1]), 
#               np.mean(accu_result[:, 2]), np.mean(accu_result[:, 3])))
print('Done!')

Image NA3777-02_AB, Test ((368, 420)), Ground Truth ((368, 420)):   0%|          | 0/30 [00:03<?, ?it/s]

Image NA4077-02_AB, Test ((360, 575)), Ground Truth ((360, 575)):   3%|▎         | 1/30 [00:10<02:59,  6.19s/it]

Image NA4092-02_AB, Test ((352, 513)), Ground Truth ((352, 513)):   7%|▋         | 2/30 [00:18<03:10,  6.80s/it]

Image NA4107-02_AB, Test ((284, 529)), Ground Truth ((284, 529)):  10%|█         | 3/30 [00:24<03:07,  6.93s/it]

Image NA4160-02_AB, Test ((326, 466)), Ground Truth ((326, 466)):  13%|█▎        | 4/30 [00:30<02:52,  6.65s/it]

...

Image NA5010-02_AB17-24, Test ((604, 591)), Ground Truth ((604, 591)):  90%|█████████ | 27/30 [02:43<00:13,  4.40s/it]

Image NA5015-02_AB17-24, Test ((315, 435)), Ground Truth ((315, 435)):  93%|█████████▎| 28/30 [02:53<00:14,  7.31s/it]

Image NA5029-02_AB17-24, Test ((340, 420)), Ground Truth ((340, 420)):  97%|█████████▋| 29/30 [02:59<00:06,  6.77s/it]

Image NA5029-02_AB17-24, Test ((340, 420)), Ground Truth ((340, 420)): 100%|██████████| 30/30 [03:01<00:00,  6.07s/it]

Done!


In [4]:
!nvidia-smi

Sun May 17 23:49:53 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN Xp            Off  | 00000000:65:00.0 Off |                  N/A |
| 31%   53C    P2    64W / 250W |   1309MiB / 12194MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  